In [1]:
import tensorflow as tf

c:\users\daniel\anaconda5.0.1-python3.6\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(tf.__version__)

1.4.0


In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x * x * y + y + 2

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [5]:
sess.close()

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [7]:
init = tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [9]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [10]:
sess.close()

In [11]:
x1 = tf.Variable(1)

In [12]:
x1.graph is tf.get_default_graph()

True

In [13]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [14]:
x2.graph is graph

True

In [15]:
x2.graph is tf.get_default_graph()

False

In [16]:
tf.reset_default_graph()

In [17]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [18]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [19]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()

In [4]:
num_trainings, num_features = housing.data.shape
print(num_trainings, num_features)

20640 8


In [5]:
housing_data_plus_bias = np.c_[np.ones((num_trainings, 1)), housing.data]

In [9]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [10]:
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7465141e+01]
 [ 4.3573415e-01]
 [ 9.3382923e-03]
 [-1.0662201e-01]
 [ 6.4410698e-01]
 [-4.2513184e-06]
 [-3.7732250e-03]
 [-4.2664889e-01]
 [-4.4051403e-01]]


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((num_trainings, 1)), scaled_housing_data]

In [12]:
tf.reset_default_graph()

n_epochs= 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([num_features + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/num_trainings * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [13]:
init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MES =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MES = 11.589013
Epoch 100 MES = 0.83702457
Epoch 200 MES = 0.6345521
Epoch 300 MES = 0.6019306
Epoch 400 MES = 0.58031756
Epoch 500 MES = 0.5647551
Epoch 600 MES = 0.55352247
Epoch 700 MES = 0.54541475
Epoch 800 MES = 0.539561
Epoch 900 MES = 0.53533494
[[ 2.0685525 ]
 [ 0.79008037]
 [ 0.14624448]
 [-0.12553659]
 [ 0.16122444]
 [ 0.00601163]
 [-0.04080889]
 [-0.7365989 ]
 [-0.69949394]]


In [15]:
tf.reset_default_graph()

n_epochs= 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([num_features + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

In [16]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MES =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MES = 14.306341
Epoch 100 MES = 0.8000337
Epoch 200 MES = 0.58584493
Epoch 300 MES = 0.5665314
Epoch 400 MES = 0.55485773
Epoch 500 MES = 0.5464552
Epoch 600 MES = 0.54037786
Epoch 700 MES = 0.5359769
Epoch 800 MES = 0.5327912
Epoch 900 MES = 0.5304828
[[ 2.0685525 ]
 [ 0.8118042 ]
 [ 0.14083818]
 [-0.18431054]
 [ 0.21762271]
 [ 0.00376296]
 [-0.04081476]
 [-0.75561035]
 [-0.7218915 ]]


In [17]:
tf.reset_default_graph()

n_epochs= 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([num_features + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [18]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MES =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MES = 6.2387834
Epoch 100 MES = 0.7799117
Epoch 200 MES = 0.6341927
Epoch 300 MES = 0.6034601
Epoch 400 MES = 0.5835989
Epoch 500 MES = 0.5690333
Epoch 600 MES = 0.55820966
Epoch 700 MES = 0.55012935
Epoch 800 MES = 0.54407364
Epoch 900 MES = 0.53951585
[[ 2.0685525 ]
 [ 0.8711893 ]
 [ 0.15659298]
 [-0.28786585]
 [ 0.2994522 ]
 [ 0.0087228 ]
 [-0.04341354]
 [-0.58498913]
 [-0.5577638 ]]


In [25]:
tf.reset_default_graph()

n_epochs= 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([num_features + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [26]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MES =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MES = 5.0608563
Epoch 100 MES = 0.53022677
Epoch 200 MES = 0.5250967
Epoch 300 MES = 0.5244237
Epoch 400 MES = 0.52433485
Epoch 500 MES = 0.5243225
Epoch 600 MES = 0.5243208
Epoch 700 MES = 0.5243205
Epoch 800 MES = 0.52432084
Epoch 900 MES = 0.52432054
[[ 2.0685577 ]
 [ 0.8296292 ]
 [ 0.11875348]
 [-0.26554582]
 [ 0.30571193]
 [-0.00450247]
 [-0.03932664]
 [-0.89986277]
 [-0.87051934]]


In [27]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [28]:
X = tf.placeholder(tf.float32, shape=(None, num_features + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

batch_size = 100
n_batches = int(np.ceil(num_trainings / batch_size))

In [7]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * num_features + batch_index)
    indices = np.random.randint(num_trainings, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

In [31]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        print(epoch)
    
    best_theta = theta.eval()
    print(best_theta)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [38]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([num_features + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name='predicitions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
            save_path = saver.save(sess, "./my_model.ckpt")
        sess.run(training_op)
        
    best_theta = theta.eval()
    print(best_theta)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

Epoch 0 MSE =  2.7544272
Epoch 100 MSE =  0.63222194
Epoch 200 MSE =  0.5727796
Epoch 300 MSE =  0.5585005
Epoch 400 MSE =  0.54906934
Epoch 500 MSE =  0.5422877
Epoch 600 MSE =  0.5373788
Epoch 700 MSE =  0.5338219
Epoch 800 MSE =  0.5312427
Epoch 900 MSE =  0.52937055
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


In [39]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    print(theta.eval())

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


In [32]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [26]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, num_features + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([num_features + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [27]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [28]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(num_trainings / batch_size))

In [33]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()

In [34]:
file_writer.close()

In [35]:
print(error.op.name)

loss_1/sub


In [8]:
tf.reset_default_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, threshold, name="relu")

X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [9]:
file_writer = tf.summary.FileWriter("tf_logs/relu9", tf.get_default_graph())
file_writer.close()

## Exercise

In [1]:
from datetime import datetime
import numpy as np
import os
from sklearn.datasets import make_moons
from sklearn.metrics import precision_score, recall_score
import tensorflow as tf

c:\users\daniel\anaconda5.0.1-python3.6\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

In [3]:
samples = 10000
noise_level = 0.1

In [4]:
X, y = make_moons(samples, noise=noise_level)

In [5]:
X_with_bias = np.c_[np.ones((samples, 1)), X]
y_column = y.reshape(-1, 1)

In [6]:
test_ratio = 0.2
test_size = int(samples * test_ratio)

In [7]:
X_train = X_with_bias[:-test_size]
y_train = y_column[:-test_size]
X_test = X_with_bias[-test_size:]
y_test = y_column[-test_size:]

In [8]:
def random_batch(X_train, y_train, batch_size):
    # Method to get random batches. Indices can be repeated within a batch
    indices = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

In [9]:
def logistic_regression(X, y, learning_rate):
    # Ensure we can handle any number of features
    num_features = int(X.get_shape()[1])
    with tf.name_scope("logistic_regression"):
        with tf.name_scope("model"):
            # The actual logistic regression step
            theta = tf.Variable(tf.random_uniform([num_features, 1], -1.0, 1.0), name="theta")
            logits = tf.matmul(X, theta)
            y_prob = tf.sigmoid(logits)
        with tf.name_scope("train"):
            # Compute the loss function and define the optimizer and training goal
            loss = tf.losses.log_loss(y, y_prob)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate)
            training_op = optimizer.minimize(loss)
            loss_summary = tf.summary.scalar("Log_Loss", loss)
        with tf.name_scope("save"):
            # Node to save the model during execution
            saver = tf.train.Saver()
        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
    return y_prob, loss, training_op, loss_summary, saver, init

In [10]:
def log_dir():
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    logdir = "{}/run-{}/".format(root_logdir, now)
    return logdir

In [11]:
# Run parameters
n_epochs = 1000
batch_size = 50
n_batches = int(np.ceil(samples / batch_size))
learning_rate = 0.01
# Checkpoint paths
checkpoint_path = "./logistic_regression.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_path = "./logistic_regression_final.ckpt"

# Make graph
X = tf.placeholder(tf.float32, shape=(None, 3), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
y_prob, loss, training_op, loss_summary, saver, init = logistic_regression(X, y, learning_rate)
# Save to a file
file_writer = tf.summary.FileWriter(log_dir(), tf.get_default_graph())

with tf.Session() as sess:
    # Check to see if the checkpoint file exists
    if os.path.isfile(checkpoint_epoch_path):
        # Set the starting epoch
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        # Load the checkpoint
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        # Start a fresh run
        start_epoch = 0
        sess.run(init)
    
    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            # Get a new batch
            X_batch, y_batch = random_batch(X_train, y_train, batch_size)
            # Run a training step
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        # Compute the current loss and add it to the summary
        loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test, y: y_test})
        file_writer.add_summary(summary_str, epoch)
    
        if epoch % 100 == 0:
            # Print progress
            print("Epoch:", epoch, "\tLoss:", loss_val)
            # Save a checkpoint file
            save_path = saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b'%d' % (epoch + 1))
    
    # Get the final probabilities
    y_prob_val = y_prob.eval(feed_dict={X: X_test, y: y_test})
    # Save the final model
    save_path = saver.save(sess, final_path)
    file_writer.close()

# Remove the checkpoint flag
os.rm(checkpoint_epoch_path)
    
# See how our predictions did
y_pred = y_prob_val >= 0.5
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))

INFO:tensorflow:Summary name Log Loss is illegal; using Log_Loss instead.
Epoch: 0 	Loss: 0.59070534
Epoch: 100 	Loss: 0.24579407
Epoch: 200 	Loss: 0.24354719
Epoch: 300 	Loss: 0.24313715
Epoch: 400 	Loss: 0.24328718
Epoch: 500 	Loss: 0.24306348
Epoch: 600 	Loss: 0.24306397
Epoch: 700 	Loss: 0.24324153
Epoch: 800 	Loss: 0.24341574
Epoch: 900 	Loss: 0.24315372
0.8744939271255061
0.8816326530612245
